In [2]:
!pip install llama-index openai PyMuPDF pandas
!pip install tqdm
!pip install transformers

In [3]:
import fitz  # PyMuPDF
import pandas as pd
import os
import random
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import time

In [14]:
txt_folder = "C:/Users/eng.baris/Desktop/New folder"

In [5]:
# T5 model and tokenizer
model_name = "ramsrigouthamg/t5_squad_v1"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def generate_questions_from_text_file(txt_file_path, tokenizer, model, num_questions=20, max_length=512):
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    questions = []
    text_length = len(text)
    
    for _ in range(num_questions):
        # Randomly select a starting point
        if text_length > max_length:
            start_idx = random.randint(0, text_length - max_length)
        else:
            start_idx = 0

        input_text = text[start_idx:start_idx + max_length]

        # Generator
        input_text = f"generate question: {input_text}"
        if not input_text.endswith("</s>"):
            input_text += " </s>"

        # Creating Questions
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
        output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
        question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Remove "question:" prefix
        question = question.replace("question:", "").strip()

        questions.append(question)

    return questions

# Collect data for the CSV file
all_questions = []

for txt_file in tqdm(os.listdir(txt_folder), desc="Generating Questions"):
    if txt_file.endswith(".txt"):
        txt_file_path = os.path.join(txt_folder, txt_file)
        questions = generate_questions_from_text_file(txt_file_path, tokenizer, model, num_questions=20)
        for question in questions:
            all_questions.append((question, txt_file))

# Create and save DataFrame
df = pd.DataFrame(all_questions, columns=["question", "path"])
df.to_csv("C:/Users/eng.baris/Desktop/questions2.csv", index=False)

print("CSV file created: questions2.csv")

Generating Questions:   0%|          | 0/5827 [00:00<?, ?it/s]